### Setup

In [ ]:
# Clone main repo
!git clone https://github.com/rossiyareich/marching-waifu-x.git
%cd marching-waifu-x

# Update submodules
!git submodule update --init --recursive

# Create directories
!mkdir data/checkpoints
!mkdir data/dataset/diffusion
!mkdir data/dataset/nerf/train
!mkdir data/dataset/nerf/test
!mkdir data/dataset/nerf_output
!mkdir data/dataset/ngp/train
!mkdir data/dataset/ngp_output
!mkdir data/dataset/ngp_output/frames

# Install requirements
!python -m pip install -r requirements.txt

# Setup GroundingDINO + SegmentAnything
%cd ext/Grounded-Segment-Anything
!python -m pip install -q -r requirements.txt
%cd GroundingDINO
!python -m pip install -q .
%cd ../segment_anything
!python -m pip install -q .
%cd ../../..

# Setup nvdiffrec
!python -m pip install ninja imageio PyOpenGL glfw xatlas gdown
!python -m pip install git+https://github.com/NVlabs/nvdiffrast/
!python -m pip install --global-option="--no-networks" git+https://github.com/NVlabs/tiny-cuda-nn#subdirectory=bindings/torch
!imageio_download_bin freeimage

# Setup InstantNGP
!apt update
!apt install build-essential git python3-dev python3-pip libopenexr-dev libxi-dev libglfw3-dev libglew-dev libomp-dev libxinerama-dev libxcursor-dev colmap ffmpeg jq
!python -m pip install --upgrade cmake
%cd ext/instant-ngp

compute_capability = "75" #@param [50, 52, 60, 61, 70, 72, 75, 80, 86, 87]
%env TCNN_CUDA_ARCHITECTURES=$compute_capability

network_type = "FullyFusedMLP" if int(compute_capability) >= 70 else "CutlassMLP"
print(f"Using {network_type}")
%env NN_CONFIG_PATH = ./configs/nerf/base.json
!jq '.network.otype = "CutlassMLP" | .rgb_network.otype = "CutlassMLP"' $NN_CONFIG_PATH | sponge $NN_CONFIG_PATH

!cmake . -B build -DNGP_BUILD_WITH_GUI=OFF
!cmake --build build --config RelWithDebInfo -j `nproc`
!python -m pip install -r requirements.txt
%cd ../../ipynb

### Imports

In [ ]:
%cd ../scripts

In [ ]:
import sys

sys.path.append("..")

import gc
import glob
import json
import os

import cv2
from huggingface_hub import notebook_login
import PIL.Image
from IPython.display import Image

from src.utils.image_wrapper import *

In [ ]:
notebook_login()

### ControlVideo

In [ ]:
def save_gif(fps, images, scale, gif_path):
    images = [image_wrapper(image, "pil").scale(scale).to_pil() for image in images]
    images[0].save(
        gif_path,
        save_all=True,
        append_images=images[1:],
        optimize=True,
        duration=len(images) / fps,
        loop=0,
    )


def display_gif(gif_path):
    display(Image(data=open(gif_path, "rb").read(), format="png"))

In [ ]:
with open("inference.json", "r") as f:
    conf = json.load(f)

In [ ]:
!python inference_controlvideo.py --settings_path "inference.json" 

In [ ]:
# Load images
images = sorted(glob.glob(os.path.join(conf["paths"]["diffusion_output_path"], "*.png")))
images = [PIL.Image.open(image) for image in images]

save_gif(
    10.0,
    images,
    1.0,
    "../ipynb/diffusion.gif",
)
del images
gc.collect()

display_gif("../ipynb/diffusion.gif")

### GroundingDINO + SegmentAnything

In [ ]:
!python inference_groundedsam.py --settings_path inference.json

In [ ]:
# Load masked images
masked_images = sorted(glob.glob(os.path.join(conf["paths"]["masked_path"], "*.png")))
masked_images = [PIL.Image.open(image) for image in images]

save_gif(
    10.0,
    masked_images,
    1.0,
    "../ipynb/masked.gif",
)
del masked_images
gc.collect()

display_gif("../ipynb/masked.gif")

In [ ]:
!cp -r ../data/dataset/ngp/train/. ../data/dataset/nerf/train
!cp -r ../data/dataset/ngp/train/. ../data/dataset/nerf/test

### nvdiffrec

In [ ]:
!cp -r ../data/dataset/nerf ../ext/nvdiffrec/data
!cp -r nvdiffrec.json ../ext/nvdiffrec/configs

In [ ]:
%cd ../ext/nvdiffrec
!python train.py --config configs/nvdiffrec.json
%cd ../../scripts
!cp -r ../ext/nvdiffrec/nerf_output/. ../data/dataset/nerf_output

### InstantNGP

In [ ]:
%cd ../ext/instant-ngp

In [ ]:
training_data = "../../data/dataset/ngp"
snapshot = "../../data/dataset/ngp_output/results.ingp"
sharpen = 1.0
n_steps = 5000

video_camera_path = "../../data/dataset/ngp/base_cam.json"
video_fps = 10
video_n_seconds = 10
video_output = "../../ipynb/ngp.mp4"
width, height = 768, 1024

In [ ]:
!python ./scripts/run.py \
    --training_data {training_data} \
    --save_snapshot {snapshot} \
    --n_steps {n_steps} \
    --sharpen {sharpen}

In [ ]:
!python ./scripts/run.py \
    {snapshot} \
    --video_camera_path {video_camera_path} \
    --video_fps {video_fps} \
    --video_n_seconds {video_n_seconds} \
    --video_output {video_output} \
    --width {width} \
    --height {height} \
    --sharpen {sharpen}

In [ ]:
%cd ../../scripts

In [ ]:
video_output_cv2 = "../ipynb/ngp.mp4"
frames_cv2 = "../data/dataset/ngp_output/frames"

vc = cv2.VideoCapture(video_output_cv2)
success, cv2_img = vc.read()
count = 0
while success:
    frame_path = os.path.join(frames_cv2, f"ngp{count}.png")
    cv2.imwrite(frame_path, cv2_img)
    success, cv2_img = vc.read()
    print(f"Saved frame {cv2_img}, {success}")
    count += 1

### Evaluation

In [ ]:
!python evaluation.py --settings_path evaluation.json --path ../data/dataset/ngp/train/*.png

In [ ]:
!python evaluation.py --settings_path evaluation.json --path ../data/dataset/ngp_output/ngp*.png

In [ ]:
!python evaluation.py --settings_path evaluation.json --path ../data/dataset/nerf_output/opt*.png